In [24]:
import os
import pickle
import numpy as np

import mlflow
from mlflow.tracking import MlflowClient

from flask import Flask, request, jsonify

import mlflow.pyfunc
from tensorflow.keras.preprocessing import text, sequence
import tensorflow as tf

from pathlib import Path

MLFLOW_TRACKING_URI=os.getenv('MLFLOW_TRACKING_URI')
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


def get_tokenizer():
    with open('./artifact/tokenizer.bin', "rb") as f_in:
        tokenizer = pickle.load(f_in)
    
    return tokenizer


def prepare(text):
    maxlen = 300
    tokenizer = get_tokenizer()
    tokens = tokenizer.texts_to_sequences(text)
    tokens = sequence.pad_sequences(tokens, maxlen=maxlen)
    return tokens
    
def load_model():
    uri_path = Path.cwd().joinpath('artifact/model').as_uri()
    model = mlflow.keras.load_model(uri_path)
    return model
    

def classify(text):
    print("Loading the model...")
    model = load_model()
    print("Successful!")
    preds = model.predict(text)
    return preds


# app = Flask('fake-news-classifier')


# @app.route('/classify', methods=['POST'])
def classify_endpoint(text):

    prepped_text = prepare(text)

    pred = classify(prepped_text)

    pred = tf.argmax(pred, axis=1).numpy()

    result = {
        'text': text,
        'class': pred
    }

    return result

def lambda_handler(event, context):
    text = event['text']
    prepped_text = prepare(text)

    pred = classify(prepped_text)

    pred = tf.argmax(pred, axis=1).numpy()[0].tolist()

    result = {
        'text': text,
        'class': pred
    }
    return result

In [29]:
import pandas as pd

true = pd.read_csv('./data/True.csv')



In [30]:
lambda_handler({'text': true.iloc[0].text}, None)

Loading the model...
Successful!
146/146 [==============================] - 27s 179ms/step


{'text': 'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that suppo

In [63]:
import os

import mlflow
from mlflow.tracking import MlflowClient

from flask import Flask, request, jsonify


import mlflow.pyfunc

MLFLOW_TRACKING_URI=os.environ['MLFLOW_TRACKING_URI']
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

model_name = "fake-news-detect-1"

model_versions = client.search_model_versions(f"name='{model_name}'")

In [64]:
model_versions

[<ModelVersion: creation_timestamp=1662027138594, current_stage='Production', description='', last_updated_timestamp=1662027803213, name='fake-news-detect-1', run_id='d92d79a7278d4408b2406d82b2b09aab', run_link='', source='s3://my-mlflow-models-bucket/d92d79a7278d4408b2406d82b2b09aab/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>,
 <ModelVersion: creation_timestamp=1662026816835, current_stage='Production', description='', last_updated_timestamp=1662027839298, name='fake-news-detect-1', run_id='d92d79a7278d4408b2406d82b2b09aab', run_link='', source='s3://my-mlflow-models-bucket/d92d79a7278d4408b2406d82b2b09aab/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>]

: 

In [3]:
mlflow.artifacts.download_artifacts(run_id = 'd92d79a7278d4408b2406d82b2b09aab', dst_path='./artifact')

'/home/thresh/MLOPS/final_mlops_zoomcamp/artifact/'

In [59]:
dict(client.get_latest_versions(name=model_name)[1])

IndexError: list index out of range

In [15]:
import pickle

In [52]:
model = mlflow.keras.load_model(uri_path)

In [10]:
import tensorflow as tf

In [13]:
model = tf.keras.models.load_model('./main/data/model')

In [41]:
from pathlib import Path

In [49]:
%cd ..

/home/thresh/MLOPS/final_mlops_zoomcamp


In [51]:
path = 'artifact/model/data/model'

uri_path = Path.cwd().joinpath('artifact/model').as_uri()
uri_path

'file:///home/thresh/MLOPS/final_mlops_zoomcamp/artifact/model'

In [45]:
uri_path

'file:///home/thresh/MLOPS/final_mlops_zoomcamp/bint/main/data/model'

In [44]:
Path.cwd().joinpath('main/data/model')

PosixPath('/home/thresh/MLOPS/final_mlops_zoomcamp/bint/main/data/model')

In [16]:
import pickle
with open('../artifact/tokenizer.bin', "rb") as f_in:
    tokenizer = pickle.load(f_in)

In [55]:
mv = client.search_model_versions("name='fake-news-detect-1'")

dict(mv[0])

{'creation_timestamp': 1662027138594,
 'current_stage': 'Production',
 'description': '',
 'last_updated_timestamp': 1662027803213,
 'name': 'fake-news-detect-1',
 'run_id': 'd92d79a7278d4408b2406d82b2b09aab',
 'run_link': '',
 'source': 's3://my-mlflow-models-bucket/d92d79a7278d4408b2406d82b2b09aab/artifacts/model',
 'status': 'READY',
 'status_message': '',
 'tags': {},
 'user_id': '',
 'version': '2'}

In [17]:
from tensorflow.keras.preprocessing import text, sequence
maxlen = 300

In [18]:
tokens = tokenizer.texts_to_sequences("i am a boy")
tokens = sequence.pad_sequences(tokens, maxlen=maxlen)

In [40]:
predicted = model.predict(tokens)

146/146 [==============================] - 27s 187ms/step


In [33]:
predicted

array([[0.02289921],
       [0.02186468],
       [0.025014  ],
       ...,
       [0.02553978],
       [0.02061237],
       [0.02061237]], dtype=float32)

In [32]:
np.where(predicted > 0.5, 1,0)

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [21]:
import numpy as np

In [30]:
classes_x=np.argmax(predicted,axis=1)

In [36]:
loc = 's3://my-mlflow-models-bucket'

In [38]:
model = mlflow.keras.load_model(f'{loc}/421f9eff7a8a4ce89842c044d9fe6aad/artifacts/model')

In [39]:
model.predict(tokens)

146/146 [==============================] - 28s 184ms/step


array([[0.0755357 ],
       [0.07814227],
       [0.07427611],
       ...,
       [0.07228513],
       [0.10534022],
       [0.10534022]], dtype=float32)

In [31]:
classes_x

array([0, 0, 0, ..., 0, 0, 0])

In [58]:
model = mlflow.keras.load_model(
    model_uri = model_uri, dst_path='./artifact'
)

In [27]:
import pandas as pd
df = pd.read_csv("../data/True.csv")

In [28]:
tokens = tokenizer.texts_to_sequences(df.iloc[0].text)
tokens = sequence.pad_sequences(tokens, maxlen=maxlen)


In [32]:
tokens

array([[   0,    0,    0, ...,    0,    0,  659],
       [   0,    0,    0, ...,    0,    0, 2527],
       [   0,    0,    0, ...,    0,    0,  241],
       ...,
       [   0,    0,    0, ...,    0,    0, 1499],
       [   0,    0,    0, ...,    0,    0,    4],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int32)

In [39]:
import inspect

In [52]:
model._model_impl.predict(tokens)

15/91 [===>..........................] - ETA: 14s

KeyboardInterrupt: 

In [40]:
inspect.getmembers(model)

[('__class__', mlflow.pyfunc.PyFuncModel),
 ('__delattr__',
  <method-wrapper '__delattr__' of PyFuncModel object at 0x7faefff1eee0>),
 ('__dict__',
  {'_model_meta': <mlflow.models.model.Model at 0x7faf5d51bd90>,
   '_model_impl': <mlflow.keras._KerasModelWrapper at 0x7faf6ddf9b80>}),
 ('__dir__', <function PyFuncModel.__dir__()>),
 ('__doc__',
  "\n    MLflow 'python function' model.\n\n    Wrapper around model implementation and metadata. This class is not meant to be constructed\n    directly. Instead, instances of this class are constructed and returned from\n    :py:func:`load_model() <mlflow.pyfunc.load_model>`.\n\n    ``model_impl`` can be any Python object that implements the `Pyfunc interface\n    <https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#pyfunc-inference-api>`_, and is\n    returned by invoking the model's ``loader_module``.\n\n    ``model_meta`` contains model metadata loaded from the MLmodel file.\n    "),
 ('__eq__', <method-wrapper '__eq__' of PyFuncM

In [ ]:
class KerasWrapper(mlflow.pyfunc.PythonModel):
  
  def __init__(self, keras_model, labelEncoder, labelDecoder, n):         
    self.keras_model = keras_model
    self.labelEncoder = labelEncoder
    self.labelDecoder = labelDecoder
    self.topn = n
    
  def load_context(self, context): 
    self.keras_model = mlflow.keras.load_model(model_uri=context.artifacts[self.keras_model], compile=False)
  
  def predict(self, context, input_data):
    scores = self.keras_model.predict(input_data)
    return scores